In [75]:
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import numpy as np
#from datetime import datetime
#from datetime import date

In [76]:
read_pars = pd.read_csv('StockReorderlocations_2022-01-21.csv',header=0,dtype=str)#parse_dates
read_pars.rename(columns={'ProductSKU':'SKU','MinimumBeforeReorder':'Par'},inplace=True)
read_pars.drop(['StockLocator','PickZones'],inplace=True,axis=1)
read_pars.Par = read_pars.Par.astype(float)
read_pars.Par = read_pars.Par.astype(int)
read_pars.ReorderQuantity = read_pars.ReorderQuantity.astype(float)
read_pars.ReorderQuantity = read_pars.ReorderQuantity.astype(int)
read_pars = read_pars.drop(columns=['Action','ReorderQuantity'])
read_pars.head()

,SKU,ProductName,Location,Par
0,086785024684,Drumshanbo Irish Gin 375ml,Oak Hill,1
1,086785024684,Drumshanbo Irish Gin 375ml,Pleasant Valley,4
2,086785024684,Drumshanbo Irish Gin 375ml,Airport,1
3,786992000609,Chairmans Spiced Rum 750ml,Warehouse,4
4,786992000609,Chairmans Spiced Rum 750ml,Oak Hill,2


In [77]:
df = pd.read_csv('AvailabilityReport_2022-01-26.csv')
df.head(2)

,Category,SKU,ProductName,Location,Bin,BatchSerialNumber,ExpiryDate,Discount,StockValue,OnHand,...,Allocated,Status,Unit,Family,StockLocator,Brand,PriceTier1,PriceTier2,PriceTier3,NextDeliveryDate
0,Non Alcoholic Beverage,777823111113,1 Water 355ml,Airport,NaN,NaN,NaN,NaN,24.1995,123.0,...,0.0,ACTIVE,Item,NaN,NaN,NaN,1.5,0.95,0.95,NaN
1,Non Alcoholic Beverage,777823111113,1 Water 355ml,Oak Hill,NaN,NaN,NaN,NaN,0.5357,1.0,...,0.0,ACTIVE,Item,NaN,NaN,NaN,1.5,0.95,0.95,NaN


In [78]:
df = df.drop(columns=['Bin','BatchSerialNumber','ExpiryDate','Discount','StockValue','VolumeOnHand','Available','OnOrder','InTransit','Allocated','Status','Unit','Family','StockLocator','Brand','PriceTier1','PriceTier2','PriceTier3','NextDeliveryDate'])
df.head(2)

,Category,SKU,ProductName,Location,OnHand
0,Non Alcoholic Beverage,777823111113,1 Water 355ml,Airport,123.0
1,Non Alcoholic Beverage,777823111113,1 Water 355ml,Oak Hill,1.0


In [79]:
ans = df.merge(right=read_pars,on=['SKU','ProductName','Location'])
ans.head()

,Category,SKU,ProductName,Location,OnHand,Par
0,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Pleasant Valley,0.0,4
1,Tequila,7503013136221,123 Organic Anejo Tequila 750ml,Warehouse,0.0,2
2,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Airport,2.0,0
3,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Oak Hill,1.0,0
4,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Pleasant Valley,0.0,1


In [93]:
ans['Shy'] = ans.Par - ans.OnHand
print(ans.head(40))
#checkstock = lambda row: 69 if len(ans[(ans.Location == 'Oak Hill') & (ans.ProductName == row.ProductName)].Shy) > 0 else 420
ans['Send'] = np.nan
for i, row in ans.iterrows():
    temp = ans[(ans.Location == 'Oak Hill') & (ans.ProductName == row.ProductName)]
    if len(temp) > 0 and temp.iloc[0].Shy*-1 > row.Shy and row.Shy >0 :
        print(i,'yes')
        if row.Shy <= temp.iloc[0].Shy*-1:
            row.Send = row.Shy
        else:
            row.Send = temp.iloc[0].Shy*-1
        
    #if len(temp) > 0 and row.Shy > 0:
    #    if(row.Shy < temp.iloc[0].OnHand):
    #        print(i,'here ',row.Shy - temp.iloc[0].OnHand)
    #    print(i,row.Shy)
    #    row['Candidate'] = temp.iloc[0].Shy * -1.5
    #else:
    #    print(i,'nope')
    #    row['Candidate'] = np.nan


#ans['Candidate'] = ans.apply(checkstock,axis=1)
ans.head(40)

                    Category            SKU  \
0                  Bar Tools  1234500000000   
1                    Tequila  7503013136221   
2                    Tequila  7503013136207   
3                    Tequila  7503013136207   
4                    Tequila  7503013136207   
5                    Tequila  7503013136207   
6                    Tequila  7503013136214   
7                    Tequila  7503013136214   
8                    Tequila  7503013136214   
9                    Tequila  7503013136214   
10                   Bubbles   805300030616   
11                   Bubbles   805300030616   
12                   Bubbles   805300030616   
13  Whiskey Bourbon/American   088004030711   
14  Whiskey Bourbon/American   088004030711   
15  Whiskey Bourbon/American   088004027971   
16  Whiskey Bourbon/American   088004027971   
17  Whiskey Bourbon/American   088004027971   
18  Whiskey Bourbon/American   088004027971   
19  Whiskey Bourbon/American   080660001159   
20  Whiskey B

6243 yes
6245 yes
6246 yes
6248 yes
6295 yes
6305 yes
6350 yes
6355 yes
6382 yes
6386 yes
6399 yes
6401 yes
6444 yes
6459 yes
6494 yes
6536 yes
6622 yes
6655 yes
6657 yes
6692 yes
6714 yes
6716 yes
6719 yes
6754 yes
6839 yes
6857 yes
6859 yes
6897 yes
6899 yes
6912 yes
6919 yes
6933 yes
6943 yes
6991 yes
6998 yes
7030 yes
7032 yes
7033 yes
7040 yes
7106 yes
7120 yes
7156 yes
7195 yes
7255 yes
7365 yes
7382 yes
7398 yes
7400 yes
7414 yes
7416 yes
7418 yes
7430 yes
7432 yes
7497 yes
7533 yes
7586 yes
7599 yes
7608 yes
7637 yes
7644 yes
7646 yes
7649 yes
7718 yes
7768 yes
7791 yes
7800 yes
7911 yes
7929 yes
7931 yes
7932 yes
8041 yes
8052 yes
8079 yes
8084 yes
8086 yes
8091 yes
8126 yes
8175 yes
8187 yes
8189 yes
8199 yes
8201 yes
8232 yes
8279 yes
8286 yes
8288 yes
8298 yes
8428 yes
8445 yes
8479 yes
8499 yes
8501 yes
8506 yes
8508 yes
8546 yes
8579 yes
8581 yes
8582 yes
8603 yes
8660 yes
8671 yes
8700 yes
8729 yes
8740 yes
8752 yes
8780 yes
8827 yes
8856 yes
8859 yes
8892 yes
8895 yes
8

,Category,SKU,ProductName,Location,OnHand,Par,Shy,Candidate,Send
0,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Pleasant Valley,0.0,4,4.0,NaN,NaN
1,Tequila,7503013136221,123 Organic Anejo Tequila 750ml,Warehouse,0.0,2,2.0,NaN,NaN
2,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Airport,2.0,0,-2.0,NaN,NaN
3,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Oak Hill,1.0,0,-1.0,NaN,NaN
4,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Pleasant Valley,0.0,1,1.0,NaN,NaN
5,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Warehouse,0.0,2,2.0,NaN,NaN
6,Tequila,7503013136214,123 Organic Reposado Tequila 750ml,Airport,0.0,0,0.0,NaN,NaN
7,Tequila,7503013136214,123 Organic Reposado Tequila 750ml,Oak Hill,1.0,0,-1.0,NaN,NaN
8,Tequila,7503013136214,123 Organic Reposado Tequila 750ml,Pleasant Valley,1.0,1,0.0,NaN,NaN
9,Tequila,7503013136214,123 Organic Reposado Tequila 750ml,Warehouse,0.0,2,2.0,NaN,NaN


In [81]:
#piv = ans.pivot(index='ProductName',columns='Location',values='OnHand')
#piv